In [4]:
from fetchbim.notion import NotionDatabase
from fetchbim.query import SharedFile

In [5]:
db_rules = SharedFile.get_all()
print("{} rules in database".format(len(db_rules)))
# Notion to Admin and back
rules = NotionDatabase.get_all('Shared Rules')
print("{} rules in Notion".format(len(rules)))

119 rules in database
121 rules in Notion


In [ ]:
shared_rules = []
for rule in rules[:1]:
#     # notion to python object
    shared_rule = SharedFile.from_notion(rule)
    attrs = shared_rule.Attributes
    print(shared_rule.Description)
    for attr in attrs:
        print("\t{}: {}".format(attr.Name, attr.Value))
    # object to DB
    # shared_rule.post()
    shared_rules.append(shared_rule)

In [ ]:
# # Clean slate in notion
SharedFile.archive_notion('Shared Rules')
SharedFile.archive_notion('Shared Attributes')

In [ ]:
# # DB data to notion
for shared_rule in shared_rules:
    shared_rule.to_notion()

### Manual sync

In [26]:

rule_data , attr_data = {"properties": {}}, {"properties": {}}
shared_files = SharedFile.get_all()
shared_files = shared_files['SharedFiles']
with tqdm(total=len(shared_files)) as pbar:
    for sf in shared_files:
        Id = sf['SharedFileId']
        shared = SharedFile.from_json(Id)
        # if shared.ParameterValue == "":
        #     print('Yahtzee')

        rule_data['properties']['Description'] = {"title": [{"text": {"content": shared.Description}}]}
        rule_data['properties']['SharedFileId'] = {"number": shared.SharedFileId}

        if shared.FamilyObjectType:
            rule_data['properties']['FamilyObjectType'] = {"select": {"name": shared.FamilyObjectType}}

        if shared.CategoryName:
            rule_data['properties']['CategoryName'] = {"rich_text": [{"text": {"content": shared.CategoryName}}]}

        if shared.ParameterName:
            rule_data['properties']['ParameterName'] = {"rich_text": [{"text": {"content": shared.ParameterName}}]}

        if shared.ParameterValue:
            rule_data['properties']['ParameterValue'] = {"rich_text": [{"text": {"content": shared.ParameterValue}}]}

        if shared.ParameterValueMatchType:
            if shared.ParameterValueMatchType == 0:
                rule_data['properties']['ParameterValueMatchType'] = {"select": {"name": "Equals"}}
            elif shared.ParameterValueMatchType == 1:
                rule_data['properties']['ParameterValueMatchType'] = {"select": {"name": "StartsWith"}}
            elif shared.ParameterValueMatchType == 2:
                rule_data['properties']['ParameterValueMatchType'] = {"select": {"name": "EndsWith"}}
            elif shared.ParameterValueMatchType == 3:
                rule_data['properties']['ParameterValueMatchType'] = {"select": {"name": "Contains"}}
        rule_data['properties']['Deleted'] = {"checkbox": shared.Deleted}


        exists_filt = notion.create_number_filter(shared.SharedFileId, database_property='SharedFileId')
        exists = notion.query_database('Shared Rules', filt=exists_filt)

        if exists:
            r = notion.update_page(exists[0]['id'], rule_data)
            share_page_id = exists[0]['id']
        else:
            r = notion.create_page('Shared Rules', rule_data)
            share_page_id = r.json()['id']

        for at in shared.Attributes:
            attr_data['properties']['SharedAttributeId'] = {"number": at.SharedAttributeId}
            if at.AttributeType == 0:
                attr_data['properties']['AttributeTypeDescription'] = {"select": {"name": "Property"}}
            elif at.AttributeType == 1:
                attr_data['properties']['AttributeTypeDescription'] = {"select": {"name": "Parameter"}}
            attr_data['properties']['Name'] = {"title": [{"text": {"content": at.Name}}]}
            attr_data['properties']['Value'] = {"rich_text": [{"text": {"content": at.Value}}]}
            attr_data['properties']['Deleted'] = {"checkbox": at.Deleted}
            if at.DataType:
                attr_data["properties"]['DataType'] = {"select": {"name": at.DataType}}
            if at.ParameterType:
                attr_data["properties"]['ParameterType'] = {"select": {"name": at.ParameterType}}
            if at.Sort:
                attr_data["properties"]['Sort'] = {"number": int(at.Sort)}
                attr_data["properties"]['Hidden'] = {"checkbox": at.Hidden}

            attr_data['properties']['SharedRule'] = {"relation": [{"id": share_page_id}]}

            exists_filt2 = notion.create_number_filter(at.SharedAttributeId, database_property='SharedAttributeId')
            exists2 = notion.query_database('Shared Attributes', filt=exists_filt2)

            if exists2:
                r = notion.update_page(exists2[0]['id'], attr_data)
                # print(r.text)
            else:
                r = notion.create_page('Shared Attributes', attr_data)
                # print(r.text)

            
        pbar.update()





KeyboardInterrupt: 